In [ ]:
String cron_string = BRANCH_NAME == "master" ? "H 12 * * 1,3,5" : ""
 
pipeline {
  agent { label 'ephemeral-linux' }
  options {
    // The Build GPU stage depends on the image from the Push CPU stage
    disableConcurrentBuilds()
  }
  triggers {
    cron(cron_string)
  }
  environment {
    GIT_COMMIT_SHORT = sh(returnStdout: true, script:"git rev-parse --short=7 HEAD").trim()
    GIT_COMMIT_SUBJECT = sh(returnStdout: true, script:"git log --format=%s -n 1 HEAD").trim()
    GIT_COMMIT_AUTHOR = sh(returnStdout: true, script:"git log --format='%an' -n 1 HEAD").trim()
    GIT_COMMIT_SUMMARY = "`<https://github.com/Kaggle/docker-python/commit/${GIT_COMMIT}|${GIT_COMMIT_SHORT}>` ${GIT_COMMIT_SUBJECT} - ${GIT_COMMIT_AUTHOR}"
    SLACK_CHANNEL = sh(returnStdout: true, script: "if [[ \"${GIT_BRANCH}\" == \"master\" ]]; then echo \"#kernelops\"; else echo \"#builds\"; fi").trim()
    PRETEST_TAG = sh(returnStdout: true, script: "if [[ \"${GIT_BRANCH}\" == \"master\" ]]; then echo \"ci-pretest\"; else echo \"${GIT_BRANCH}-pretest\"; fi").trim()
    STAGING_TAG = sh(returnStdout: true, script: "if [[ \"${GIT_BRANCH}\" == \"master\" ]]; then echo \"staging\"; else echo \"${GIT_BRANCH}-staging\"; fi").trim()
  }
 
  stages {
    stage('Docker CPU Build') {
      options {
        timeout(time: 90, unit: 'MINUTES')
      }
      steps {
        sh '''#!/bin/bash
          set -exo pipefail
 
          ./build | ts
          ./push ${PRETEST_TAG}
        '''
      }
    }
 
    stage('Test CPU Image') {
      options {
        timeout(time: 5, unit: 'MINUTES')
      }
      steps {
        sh '''#!/bin/bash
          set -exo pipefail
 
          date
          ./test --image gcr.io/kaggle-images/python:${PRETEST_TAG}
        '''
      }
    }
 
    stage('Push CPU Image') {
      steps {
        sh '''#!/bin/bash
          set -exo pipefail
 
          date
          docker pull gcr.io/kaggle-images/python:${PRETEST_TAG}
          ./push --source-image gcr.io/kaggle-images/python:${PRETEST_TAG} ${STAGING_TAG}
        '''
      }
    }
    
    stage('Docker GPU Build') {
      // A GPU is not required to build this image. However, in our current setup,
      // the default runtime is set to nvidia (as opposed to runc) and there
      // is no option to specify a runtime for the `docker build` command.
      //
      // TODO(rosbo) don't set `nvidia` as the default runtime and use the
      // `--runtime=nvidia` flag for the `docker run` command when GPU support is needed.
      agent { label 'ephemeral-linux-gpu' }
      options {
        timeout(time: 45, unit: 'MINUTES')
      }
      steps {
        sh '''#!/bin/bash
          set -exo pipefail
          # Remove images (dangling or not) created more than 120h (5 days ago) to prevent disk from filling up.
          docker image prune --all --force --filter "until=120h" --filter "label=kaggle-lang=python"
          # Remove any dangling images (no tags).
          # All builds for the same branch uses the same tag. This means a subsequent build for the same branch
          # will untag the previously built image which is safe to do. Builds for a single branch are performed
          # serially.
          docker image prune -f
          ./build --gpu --base-image-tag ${STAGING_TAG} | ts
          ./push --gpu ${PRETEST_TAG}
        '''
      }
    }
 
    stage('Test GPU Image') {
      agent { label 'ephemeral-linux-gpu' }
      options {
        timeout(time: 20, unit: 'MINUTES')
      }
      steps {
        sh '''#!/bin/bash
          set -exo pipefail
 
          date
          ./test --gpu --image gcr.io/kaggle-private-byod/python:${PRETEST_TAG}
        '''
      }
    }
 
    stage('Push GPU Image') {
      agent { label 'ephemeral-linux-gpu' }
      steps {
        sh '''#!/bin/bash
          set -exo pipefail
 
          date
          docker pull gcr.io/kaggle-private-byod/python:${PRETEST_TAG}
          ./push --source-image gcr.io/kaggle-private-byod/python:${PRETEST_TAG} --gpu ${STAGING_TAG}
        '''
      }
    }
 
    stage('Package Versions') {
      parallel {
        stage('CPU Diff') {
          steps {
            sh '''#!/bin/bash
            ./diff
          '''
          }
        }
        stage('GPU Diff') {
          agent { label 'ephemeral-linux-gpu' }
          steps {
            sh '''#!/bin/bash
            ./diff --gpu
          '''
          }
        }
      }
    }
  }
 
  post {
    failure {
      slackSend color: 'danger', message: "*<${env.BUILD_URL}console|${JOB_NAME} failed>* ${GIT_COMMIT_SUMMARY} @kernels-backend-ops", channel: env.SLACK_CHANNEL
    }
    success {
      slackSend color: 'good', message: "*<${env.BUILD_URL}console|${JOB_NAME} passed>* ${GIT_COMMIT_SUMMARY}", channel: env.SLACK_CHANNEL
    }
    aborted {
      slackSend color: 'warning', message: "*<${env.BUILD_URL}console|${JOB_NAME} aborted>* ${GIT_COMMIT_SUMMARY}", channel: env.SLACK_CHANNEL
    }
  }
}


In [ ]:
import unittest
 
class TestImport(unittest.TestCase):
    # Basic import tests for packages without any.
    def test_basic(self):
        import bq_helper
        import cleverhans
        import tensorflow_gcs_config
 

In [ ]:
{# 
Jinja template to inject notebook cell metadata to enhance generated HTML output
All cell metadata starting with '_kg_' will be included with its value ({key}-{value}) 
as a class in the cell's DIV container
#}
    
{% extends 'full.tpl'%}
{% block any_cell %}
    <div class="{% for k in cell['metadata'] if k.startswith("_kg_") %}{{k}}-{{cell['metadata'][k] | lower}} {% endfor %}">
        {{ super() }}
    </div>
{% endblock any_cell %}

# Preços de Casas: Dados Diferentes, Mesmo Problema!

This is the same problem (but different location) from https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview 
Thus, we would like to do same approach in data exploration. 


![](https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png)

Well, this is one of the famous city, 

![](https://i0.wp.com/www.mobileworldlive.com/wp-content/uploads/2019/12/Brazil.png?fit=650%2C400&ssl=1)

In my heart, I always want to go to Brazil, working there, and watch some football games there. I always on the Seleção side on World Cup since I never saw my country there. 

![](https://i.pinimg.com/originals/21/ac/8f/21ac8f9241bff1948ff1d7c47ee74bc3.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('/kaggle/input/brasilian-houses-to-rent/houses_to_rent.csv')

In [ ]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df.head()

# Turn the "Sem Info" and "Incluso" to "0"

We turn it to string to avoid attribute error

In [ ]:
df['hoa'] = df['hoa'].replace('Sem info','0')
df['hoa'] = df['hoa'].replace('Incluso','0')

In [ ]:
df['property tax'] = df['property tax'].replace('Sem info','0')
df['property tax'] = df['property tax'].replace('Incluso','0')

In [ ]:
df['hoa'].value_counts()

In [ ]:
df['rent amount'].value_counts()

In [ ]:
df['property tax'].value_counts()

In [ ]:
df['fire insurance'].value_counts()

In [ ]:
df['total'].value_counts()

## Cleaning the Real Sign (R$)

After that, we note that the hoa, rent amount, prop tax, fire insurance, and total is in Brazilian Real.

In [ ]:
def extract_value_from(Value):
    out = Value.replace('R$', '')
    out_ = out.replace(',', '')
    out_ = float(out_)
    return out_

In [ ]:
df['hoa'] = df['hoa'].apply(lambda x: extract_value_from(x))
df['rent amount'] = df['rent amount'].apply(lambda x: extract_value_from(x))
df['property tax'] = df['property tax'].apply(lambda x: extract_value_from(x))
df['fire insurance'] = df['fire insurance'].apply(lambda x: extract_value_from(x))
df['total'] = df['total'].apply(lambda x: extract_value_from(x))

In [ ]:
df.head()

# Total Price Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.describe()

In [ ]:
print("Skewness: ", df['total'].skew())
print("Kurtosis: ", df['total'].kurt())

Okay then, we need to remove outliers using interquartile score

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

### Rows with total price more than 5622.5 are removed

In [ ]:
df = df[df['total']<= 5622.5]

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
sns.kdeplot(df['total'])
plt.title('Total Price KDE')

## What we got from total price:
1. Positive skewness
2. Deviate from normal distribution

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
sns.kdeplot(df['fire insurance'], color = 'r')
plt.title('Fire Insurance Price KDE')

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
sns.kdeplot(df['property tax'], color = 'g')
plt.title('Property Tax Price KDE')

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
sns.kdeplot(df['rent amount'], color = 'c')
plt.title('Rent Price KDE')

In [ ]:
plt.figure(figsize=(10,10))
sns.set()
sns.kdeplot(df['hoa'], color = 'y')
plt.title('HOA Price KDE')

## Total price and other price component

In [ ]:
sns.jointplot(df['total'], df['fire insurance'], kind="hex", color="r")

In [ ]:
sns.jointplot(df['total'], df['property tax'], kind="hex", color="g")

In [ ]:
sns.jointplot(df['total'], df['rent amount'], kind="hex", color="b")

In [ ]:
sns.jointplot(df['total'], df['hoa'], kind="hex", color="y")

# Relation

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="city", y="total", palette=["m", "g"], data=df)
plt.title('City and Total Price')

City 1 tend to be more expensive

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="rooms", y="total", palette=["r", "b"], data=df)
plt.title('Room and Total Price')

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="bathroom", y="total", palette=["c", "gold"], data=df)
plt.title('Bathroom and Total Price')

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="parking spaces", y="total", palette=["m", "silver"], data=df)
plt.title('Parking Spaces and Total Price')

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="floor", y="total", palette=["m", "silver"], data=df)
plt.title('Floor and Total Price')

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="animal", y="total", palette=["m", "silver"], data=df)
plt.title('Animal and Total Price')

Animal permitted tend to be more expensive

# Correlation

In [ ]:
cor = df.corr()
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(cor, annot=True, square=True);